In [1]:
import random
import numpy as np
import csv

from tqdm import tqdm

import torch
import torch.nn as nn
from torch import optim
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import torchtext
from torchtext.vocab import build_vocab_from_iterator

from TorchTransformer import *
from evaluation import *

In [2]:
# GLOBAL VARIABLES
PAD_IDX = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
MAX_LENGTH = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Set which dataset you want to run the experiment on with the DATSET variable, the name should be the same as the filename found in the atomic_datasets folder without the fileformat.

In [ ]:
DATASET = "all_dataset"

In [3]:
def yield_tokens(lines):
    for line in lines:
        yield line.split()

def create_vocab(lang):
    vocab = build_vocab_from_iterator(yield_tokens(lang), specials=["<PAD>", "<SOS>", "<EOS>"], special_first=True)
    vocab.set_default_index(-1)
    return vocab

def read_data(dataset="all"):
    print("Reading lines...")

    # Read the file and split into lines
    lines = []
    with open('atomic_datasets/' + dataset + '.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for row in reader:
            lines.append(row)
    
    lang1 = list(map(lambda x: x[0], lines))
    lang2 = list(map(lambda x: x[1], lines))

    return lang1, lang2, lines

def add_sequence_tokens(sentence):
    new_sentence = sentence.split()
    n_tokens = len(new_sentence)
    new_sentence.insert(0, "<SOS>")
    new_sentence.append("<EOS>")
    assert n_tokens + 2 == len(new_sentence)
    return " ".join(new_sentence)

def add_sequence_tokens_dataset(data):
    new_data = []
    for d in data:
        new_data.append(add_sequence_tokens(d))
    return new_data

def tensor_from_sentence(sentence, vocab):
    encoded = [vocab[word] for word in sentence.split()]
    return torch.tensor(encoded, dtype=torch.long, device=device)

def filter_pairs(pairs, MAX_LENGTH):
    filtered_pairs = []
    for s1, s2 in pairs:
        if len(s1.split()) < MAX_LENGTH and len(s2.split()) < MAX_LENGTH:
            filtered_pairs.append([s1, s2])
            
    lang1 = list(map(lambda x: x[0], filtered_pairs))
    lang2 = list(map(lambda x: x[1], filtered_pairs))
    return lang1, lang2, filtered_pairs

def pad_dataset(data, target_length):
    new_data = [] 
    for d in data:
        x = d.split()
        x = x + ["<PAD>"] * (target_length - len(x))
        new_data.append(" ".join(x))
    return new_data

def create_dataset(src_lang, src_vocab, trg_lang, trg_vocab, MAX_LENGTH):
    src_list = []
    trg_list = []
    
    src_lang = pad_dataset(src_lang, MAX_LENGTH)
    trg_lang = pad_dataset(trg_lang, MAX_LENGTH)
    
    for src, trg in zip(src_lang, trg_lang):
        src_list.append(tensor_from_sentence(src, src_vocab).to(device))
        trg_list.append(tensor_from_sentence(trg, trg_vocab).to(device))
    src_tensors = torch.stack(src_list)
    trg_tensors = torch.stack(trg_list)
    return TensorDataset(src_tensors, trg_tensors)

def create_dataset_new(src_lang, src_vocab, trg_lang, trg_vocab, MAX_LENGTH):
    src_list = []
    trg_list = []
    
    src_lang = add_sequence_tokens_dataset(src_lang)
    trg_lang = add_sequence_tokens_dataset(trg_lang)
    
    src_lang = pad_dataset(src_lang, MAX_LENGTH + 2)
    trg_lang = pad_dataset(trg_lang, MAX_LENGTH + 2)
    
    for src, trg in zip(src_lang, trg_lang):
        src_list.append(tensor_from_sentence(src, src_vocab).to(device))
        trg_list.append(tensor_from_sentence(trg, trg_vocab).to(device))
    src_tensors = torch.stack(src_list)
    trg_tensors = torch.stack(trg_list)
    return TensorDataset(src_tensors, trg_tensors)

In [4]:
_, _, atomic_pairs = read_data(DATASET)
text, logic, _ = filter_pairs(atomic_pairs, MAX_LENGTH)
text_vocab = create_vocab(text)
logic_vocab = create_vocab(logic)

print(len(text), len(logic))
print(len(text_vocab), print(len(logic_vocab)))

Reading lines...
["personx returns to personx's work xintent to keep their job", 'person (x) & returns to (x,z) & work (z) -> to keep their job (x)']
["personx returns to personx's work xintent to keep their job", 'person (x) & returns to (x,z) & work (z) -> to keep their job (x)']
571793 571793
25943
25950 None


In [5]:
total_dataset = create_dataset_new(text, text_vocab, logic, logic_vocab, MAX_LENGTH)
train_set, val_30k = torch.utils.data.random_split(total_dataset, [len(total_dataset) - 30000, 30000])
train_20k, _ = torch.utils.data.random_split(train_set, [20000, len(train_set) - 20000])
train_10k, _ = torch.utils.data.random_split(train_set, [10000, len(train_set) - 10000])
train_5k, _ = torch.utils.data.random_split(train_set, [5000, len(train_set) - 5000])
train_2k, _ = torch.utils.data.random_split(train_set, [2000, len(train_set) - 2000])


In [6]:
loader_val_30k = DataLoader(val_30k, batch_size=128)
loader_20k = DataLoader(train_20k, batch_size=128)
loader_10k = DataLoader(train_10k, batch_size=128)
loader_5k = DataLoader(train_5k, batch_size=128)
loader_2k = DataLoader(train_2k, batch_size=128)

training_loaders = [loader_2k,
                    loader_5k,
                   loader_10k,
                   loader_20k]

In [7]:
src_vocab_size = len(text_vocab)
trg_vocab_size = len(logic_vocab)
embed_dim=512
transformer2k = Transformer(src_vocab_size,
                          trg_vocab_size,
                          embed_size=embed_dim,
                          max_length=MAX_LENGTH+2,
                          dropout=0.1,
                          pad_idx=PAD_IDX).to(device)
transformer5k = Transformer(src_vocab_size,
                          trg_vocab_size,
                          embed_size=embed_dim,
                          max_length=MAX_LENGTH+2,
                          dropout=0.1,
                          pad_idx=PAD_IDX).to(device)
transformer10k = Transformer(src_vocab_size,
                          trg_vocab_size,
                          embed_size=embed_dim,
                          max_length=MAX_LENGTH+2,
                          dropout=0.1,
                          pad_idx=PAD_IDX).to(device)
transformer20k = Transformer(src_vocab_size,
                          trg_vocab_size,
                          embed_size=embed_dim,
                          max_length=MAX_LENGTH+2,
                          dropout=0.1,
                          pad_idx=PAD_IDX).to(device)

transformers = [transformer2k,
                transformer5k,
                transformer10k,
                transformer20k]

transformer_names = [DATASET + "_2k",
                     DATASET + "_5k",
                     DATASET + "_10k",
                     DATASET + "_20k"]

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(transformer2k):,} trainable parameters')

The model has 84,071,767 trainable parameters


In [8]:
class Scheduler(_LRScheduler):
    def __init__(self, 
                 optimizer: Optimizer,
                 dim_embed: int,
                 warmup_steps: int,
                 last_epoch: int=-1,
                 verbose: bool=False) -> None:

        self.dim_embed = dim_embed
        self.warmup_steps = warmup_steps
        self.num_param_groups = len(optimizer.param_groups)

        super().__init__(optimizer, last_epoch, verbose)
        
    def get_lr(self) -> float:
        lr = calc_lr(self._step_count, self.dim_embed, self.warmup_steps)
        return [lr] * self.num_param_groups


def calc_lr(step, dim_embed, warmup_steps):
    return dim_embed**(-0.5) * min(step**(-0.5), step * warmup_steps**(-1.5))

class TranslationLoss(nn.Module):
    def __init__(self, label_smoothing: float=0.0) -> None:
        super().__init__()
        self.loss_func = nn.CrossEntropyLoss(ignore_index    = PAD_IDX,
                                             label_smoothing = label_smoothing)

    def forward(self, logits: Tensor, labels: Tensor) -> Tensor:
        vocab_size = logits.shape[-1]
        logits = logits.reshape(-1, vocab_size)
        labels = labels.reshape(-1).long()
        return self.loss_func(logits, labels)

In [9]:
def train(model: nn.Module,
          loader: DataLoader,
          loss_func: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          scheduler: torch.optim.lr_scheduler._LRScheduler) -> float:

    model.train() # train mode
    
    total_loss = 0
    num_batches = len(loader)

    for source, target in tqdm(loader):
        # feed forward
        logits = model(source[:, 1:], target[:, :-1]) #input lacking EOS

        # loss calculation
        loss = loss_func(logits, target[:, 1:]) #labels lacking SOS
        total_loss += loss.item()

        # back-prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # learning rate scheduler
        if scheduler is not None:
            scheduler.step()

    # average training loss
    avg_loss = total_loss / num_batches
    return avg_loss

In [10]:
for transformer, data_loader, name in zip(transformers, training_loaders, transformer_names):
    optimizer = optim.Adam(transformer.parameters(),
                       betas=(0.9, 0.98),
                       eps=1.0e-9)
    criterion = TranslationLoss(label_smoothing=0.1)
    scheduler = Scheduler(optimizer=optimizer, dim_embed=embed_dim, warmup_steps=4000)
    print("Training", name, ":")
    best_res = float("inf")
    for e in range(30):
        res = train(transformer, data_loader, criterion, optimizer, scheduler)
        print(res)
        if res < best_res:
            best_res = res
        else:
            break
    print("------Training complete-----")
    
    torch.save(transformer.state_dict(), "./models/" + name + ".pt")


Training all_2k :


100%|███████████████████████████████████████████| 16/16 [00:14<00:00,  1.07it/s]


10.148587822914124


100%|███████████████████████████████████████████| 16/16 [00:03<00:00,  5.17it/s]


9.099200248718262


...

1.3771297498873085


100%|█████████████████████████████████████████| 157/157 [00:31<00:00,  5.00it/s]


1.3763046188718955


100%|█████████████████████████████████████████| 157/157 [00:31<00:00,  5.01it/s]


1.3768620673258594
------Training complete-----


In [14]:
def evaluation(model, val_dataset, src_vocab, trg_vocab):
    src_itos = src_vocab.get_itos()
    trg_itos = trg_vocab.get_itos()
    
    model.eval()
    
    reference_tokens = []
    predicted_tokens = []
    for source, target in tqdm(val_dataset):
        logits = model(source[1:].unsqueeze(0), target[:-1].unsqueeze(0)) #input lacking EOS
        golden = [trg_itos[t] for t in target if t != PAD_IDX][1:]
        reference_tokens.append(golden)
        target_tokens = []
        for word in logits.tolist()[-1]:
            guess = trg_itos[np.argmax(word)]
            target_tokens.append(guess)
            if guess == "<EOS>":
                break
        predicted_tokens.append(target_tokens)
    
    formula = average_formula_accuracy(predicted_tokens, reference_tokens, write_results=True)
    token = average_token_accuracy(predicted_tokens, reference_tokens)
    edit_distance = average_ld(predicted_tokens, reference_tokens)

    return formula, token, edit_distance


In [15]:
for transformer in transformers:
    print("Evaluating model:")
    formula, token, edit_distance = evaluation(transformer, val_30k, text_vocab, logic_vocab)
    print("average formula accuracy: ", formula)
    print("average token accuracy: ", token)
    print("average edit distance: ", edit_distance)
    print("-----------------------")


Evaluating model:


100%|█████████████████████████████████████| 30000/30000 [26:02<00:00, 19.20it/s]
30000it [00:05, 5472.27it/s]


average formula accuracy:  3.3333333333333335e-05
average token accuracy:  0.7383611677865762
average edit distance:  4.211566666666666
-----------------------
Evaluating model:


100%|█████████████████████████████████████| 30000/30000 [26:13<00:00, 19.06it/s]
30000it [00:05, 5481.69it/s]


average formula accuracy:  0.09436666666666667
average token accuracy:  0.8785327710245566
average edit distance:  1.9728666666666668
-----------------------
Evaluating model:


100%|█████████████████████████████████████| 30000/30000 [26:45<00:00, 18.68it/s]
30000it [00:05, 5351.08it/s]


average formula accuracy:  0.6861333333333334
average token accuracy:  0.9749083070138329
average edit distance:  0.3900666666666667
-----------------------
Evaluating model:


100%|█████████████████████████████████████| 30000/30000 [27:46<00:00, 18.01it/s]
30000it [00:05, 5333.35it/s]

average formula accuracy:  0.8076666666666666
average token accuracy:  0.9859253554849771
average edit distance:  0.22463333333333332
-----------------------
